In [31]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# Enhanced styling for publication-quality figures
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "DejaVu Serif"],
    "font.size": 16,
    "axes.labelsize": 16,
    "axes.titlesize": 16,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "legend.fontsize": 16,
    "figure.titlesize": 16,
    "figure.dpi": 300,
    "savefig.dpi": 300,
    "savefig.bbox": "tight",
    "savefig.pad_inches": 0.1,
    "lines.linewidth": 1.5,
    "axes.linewidth": 1.0,
    "grid.linewidth": 0.5,
    "xtick.major.width": 1.0,
    "ytick.major.width": 1.0,
})

# Color palette suitable for publications (colorblind-friendly)
colors = {
    'blue': '#1f77b4',
    'orange': '#ff7f0e',
    'green': '#2ca02c',
    'red': '#d62728',
    'purple': '#9467bd',
    'brown': '#8c564b',
    'pink': '#e377c2',
    'gray': '#7f7f7f',
    'yellow': '#bcbd22',
    'cyan': '#17becf'
}

def ellipsoid_points_from_axes(axes_lengths, R=np.eye(3), n_th=60, n_ph=30):
    a, b, c = axes_lengths
    th = np.linspace(0, 2*np.pi, n_th)
    ph = np.linspace(0, np.pi, n_ph)
    th, ph = np.meshgrid(th, ph)
    x0 = a * np.cos(th) * np.sin(ph)
    y0 = b * np.sin(th) * np.sin(ph)
    z0 = c * np.cos(ph)
    P = np.stack([x0, y0, z0], axis=-1)
    P_rot = P @ R.T
    X, Y, Z = P_rot[...,0], P_rot[...,1], P_rot[...,2]
    return X, Y, Z

def ellipse_points_from_matrix_2d(A, n=360):
    vals, vecs = np.linalg.eigh(A)
    axes = np.sqrt(vals)
    t = np.linspace(0, 2*np.pi, n)
    circ = np.stack([np.cos(t), np.sin(t)], axis=0)
    E = vecs @ (axes[:,None] * circ)
    return E[0], E[1]

def ellipsoid_matrix_from_J(J):
    return J @ J.T

def axes_and_rotation_from_A(A):
    vals, vecs = np.linalg.eigh(A)
    axes = np.sqrt(vals)
    return axes, vecs

# ----------------------- HAMR: 3x3 Jacobian -----------------------

def J_hamr(psi, r_w, a, b):
    c, s = np.cos(psi), np.sin(psi)
    J = np.array([
        [ (r_w/2)*( c - s*(b/a) ),  (r_w/2)*( c + s*(b/a) ),  0.0 ],
        [ (r_w/2)*( s + c*(b/a) ),  (r_w/2)*( s - c*(b/a) ),  0.0 ],
        [  (r_w/(2*a))           , -(r_w/(2*a))            ,  1.0 ]
    ])
    return J

# ----------------------- COMPA: 5D conceptual J -----------------------

def J_compa_conceptual(sigmas, coupling=0.15, seed=42):
    rng = np.random.default_rng(seed)
    A = rng.standard_normal((5,5)); U,_ = np.linalg.qr(A)
    B = rng.standard_normal((5,5)); V,_ = np.linalg.qr(B)
    S = np.diag(sigmas)

    R = np.eye(5)
    for j in (3,4):
        theta = coupling
        c, s = np.cos(theta), np.sin(theta)
        R_block = np.eye(5)
        R_block[[2,2,j,j],[2,j,2,j]] = c, -s, s, c
        R = R_block @ R
    Uc = U @ R

    J = Uc @ S @ V.T
    return J

# ----------------------- Enhanced Plotting -----------------------

def plot_hamr_me(save_path):
    psi = np.deg2rad(30.0)
    r_w, a, b = 0.1, 0.2, 0.2
    J = J_hamr(psi, r_w, a, b)
    A = ellipsoid_matrix_from_J(J)
    axes, R = axes_and_rotation_from_A(A)

    X, Y, Z = ellipsoid_points_from_axes(axes, R, n_th=120, n_ph=60)
    
    # Create figure with appropriate size
    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot with improved styling
    surf = ax.plot_surface(X, Y, Z, alpha=0.5, linewidth=0.5, 
                          edgecolor='k', color=colors['blue'], 
                          rstride=2, cstride=2, antialiased=True)
    
    # Axis labels with LaTeX formatting and increased padding
    ax.set_xlabel(r'$\dot{x}$', labelpad=15)
    ax.set_ylabel(r'$\dot{y}$', labelpad=15)
    ax.set_zlabel(r'$\dot{\psi}$', labelpad=15)
    
    # Set consistent axis limits
    ax.set_xlim([-1.0, 1.0])
    ax.set_ylim([-1.0, 1.0])
    ax.set_zlim([-1.0, 1.0])
    
    # Add grid with subtle styling
    ax.grid(True, alpha=0.3, color='gray', linestyle='-', linewidth=0.5)
    
    # Reduce number of ticks on each axis
    ax.set_xticks(np.linspace(-1.0, 1.0, 5))
    ax.set_yticks(np.linspace(-1.0, 1.0, 5))
    ax.set_zticks(np.linspace(-1.0, 1.0, 5))
    
    # Adjust viewing angle to better show all axes
    ax.view_init(elev=20, azim=30)
    
    # Add a light source to improve 3D perception
    surf.set_facecolors(surf._facecolors * 0.9 + 0.1)
    
    # Adjust the position of the plot within the figure to ensure all labels are visible
    plt.subplots_adjust(left=0, right=0.95, bottom=0, top=1)
    
    fig.tight_layout()
    fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)  # Close to avoid displaying in notebook environments

def plot_compa_projections(save_path_xyz, save_path_rp, nominal=True):
    if nominal:
        sigmas = np.array([1.0, 0.8, 0.6, 0.9, 0.9])
    else:
        sigmas = np.array([0.9, 0.7, 0.45, 0.6, 0.6])
    J5 = J_compa_conceptual(sigmas, coupling=0.2 if not nominal else 0.12,
                            seed=7 if not nominal else 5)
    A5 = J5 @ J5.T

    # Projection: [dx, dy, yaw]
    S_xyz = np.zeros((3,5)); S_xyz[0,0]=1; S_xyz[1,1]=1; S_xyz[2,2]=1
    A_xyz = S_xyz @ A5 @ S_xyz.T
    axes_xyz, R_xyz = axes_and_rotation_from_A(A_xyz)
    X, Y, Z = ellipsoid_points_from_axes(axes_xyz, R_xyz, n_th=120, n_ph=60)
    
    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot with improved styling
    surf = ax.plot_surface(X, Y, Z, alpha=0.5, linewidth=0.5, 
                          edgecolor='k', color=colors['green'], 
                          rstride=2, cstride=2, antialiased=True)
    
    # Increase label padding for better visibility
    ax.set_xlabel(r'$\dot{x}$', labelpad=15)
    ax.set_ylabel(r'$\dot{y}$', labelpad=15)
    ax.set_zlabel(r'$\dot{\psi}$', labelpad=15)
    
    ax.set_xlim([-1.0, 1.0])
    ax.set_ylim([-1.0, 1.0])
    ax.set_zlim([-1.0, 1.0])
    
    ax.grid(True, alpha=0.3, color='gray', linestyle='-', linewidth=0.5)
    
    ax.set_xticks(np.linspace(-1.0, 1.0, 5))
    ax.set_yticks(np.linspace(-1.0, 1.0, 5))
    ax.set_zticks(np.linspace(-1.0, 1.0, 5))
    
    # Adjust viewing angle to better show all axes
    ax.view_init(elev=20, azim=30)
    
    surf.set_facecolors(surf._facecolors * 0.9 + 0.1)
    
    # Adjust the position of the plot within the figure
    plt.subplots_adjust(left=0, right=0.95, bottom=0, top=1)
    
    fig.tight_layout()
    fig.savefig(save_path_xyz, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)

    # Projection: [roll, pitch]
    S_rp = np.zeros((2,5)); S_rp[0,3]=1; S_rp[1,4]=1
    A_rp = S_rp @ A5 @ S_rp.T
    x_e, y_e = ellipse_points_from_matrix_2d(A_rp, n=720)
    
    fig2, ax2 = plt.subplots(1, 1, figsize=(5, 4))
    
    # Plot ellipse with improved styling
    ax2.plot(x_e, y_e, linewidth=2, color=colors['red'])
    ax2.set_aspect('equal', 'box')
    ax2.set_xlabel(r'$\dot{\phi}$', labelpad=5)
    ax2.set_ylabel(r'$\dot{\theta}$', labelpad=5)
    
    # Add grid with subtle styling
    ax2.grid(True, alpha=0.3, color='gray', linestyle='-', linewidth=0.5)
    
    # Set axis limits with some padding
    x_range = np.max(np.abs(x_e)) * 1.1
    y_range = np.max(np.abs(y_e)) * 1.1
    ax2.set_xlim(-x_range, x_range)
    ax2.set_ylim(-y_range, y_range)
    
    # Reduce number of ticks
    ax2.locator_params(axis='x', nbins=5)
    ax2.locator_params(axis='y', nbins=5)
    
    fig2.tight_layout()
    fig2.savefig(save_path_rp, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig2)

# ----------------------- Run and save -----------------------

if __name__ == "__main__":
    plot_hamr_me('hamr_ellipsoid.png')
    plot_compa_projections('nominal_xyz.png', 'nominal_rp.png', nominal=True)
    # plot_compa_projections('loaded_xyz.png', 'loaded_rp.png', nominal=False)

    print("Saved figures:")
    print("- hamr_ellipsoid.png")
    print("- nominal_xyz.png")
    print("- nominal_rp.png")
    # print("- loaded_xyz.png")
    # print("- loaded_rp.png")

Saved figures:
- hamr_ellipsoid.png
- nominal_xyz.png
- nominal_rp.png
